In [2]:
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath('..'))

In [3]:
df_merged = pd.read_parquet("../data/final/test_candidates_raw.parquet").drop('target_rank', axis=1)

In [4]:
import pandas as pd
import numpy as np
from joblib import load
from loguru import logger
from app.utils.embedding_utils import explode_embeddings
from app.stages.feature_engineering_stage import calculate_area_similarity
from app.utils.strings_clean_utils import clean_area_atuacao

def process_new_data(new_data, pipeline_path="../models/feature_pipeline.joblib"):
    """
    Processa novos dados usando o pipeline de features salvo.
    
    Args:
        new_data (pd.DataFrame): DataFrame com os novos dados a serem processados
        pipeline_path (str): Caminho para o pipeline salvo
        
    Returns:
        scipy.sparse.csr_matrix: Dados processados prontos para o modelo
    """
    # 1. Carrega o pipeline
    logger.info(f"Carregando pipeline de {pipeline_path}...")
    pipe = load(pipeline_path)
    
    # 2. Faz uma cópia para não modificar o DataFrame original
    df_processed = new_data.copy()
    
    # 3. Aplica os mesmos pré-processamentos feitos em apply_feature_pipeline
    logger.info("Aplicando transformações iniciais...")
    
    # 3.1 Expande embeddings
    df_processed = explode_embeddings(df_processed)
    
    # 3.2 Preenche valores faltantes
    for col in df_processed.columns:
        if pd.api.types.is_numeric_dtype(df_processed[col]):
            df_processed[col] = df_processed[col].fillna(-999)
        else:
            df_processed[col] = df_processed[col].fillna("Indefinido")
    
    # 3.3 Limpa colunas de área de atuação
    df_processed['candidato_area_atuacao'] = clean_area_atuacao(df_processed, 'candidato_area_atuacao')
    df_processed['vaga_areas_atuacao_clean'] = clean_area_atuacao(df_processed, 'vaga_areas_atuacao')
    
    # 3.4 Calcula similaridade de área
    df_processed = calculate_area_similarity(df_processed)
    
    # 4. Aplica as transformações do pipeline
    logger.info("Aplicando transformações do pipeline...")
    X_processed = pipe.transform(df_processed)
    
    logger.success("Dados processados com sucesso!")
    return X_processed

In [5]:
X_novos = process_new_data(df_merged)

2025-06-19 09:42:37.046 | INFO     | __main__:process_new_data:21 - Carregando pipeline de ../models/feature_pipeline.joblib...
2025-06-19 09:42:37.052 | INFO     | __main__:process_new_data:28 - Aplicando transformações iniciais...
/tmp/ipykernel_23628/73994828.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_processed['vaga_areas_atuacao_clean'] = clean_area_atuacao(df_processed, 'vaga_areas_atuacao')
2025-06-19 09:42:37.865 | INFO     | __main__:process_new_data:48 - Aplicando transformações do pipeline...
2025-06-19 09:42:37.894 | SUCCESS  | __main__:process_new_data:51 - Dados processados com sucesso!


In [42]:
predicoes = modeleao.predict(X_novos)

In [43]:
predicoes

array([0.9789077 , 1.28549868, 0.31257997, ..., 0.55091626, 1.0716712 ,
       0.70600024], shape=(4273,))

In [40]:
import pandas as pd
import numpy as np
from joblib import load
from loguru import logger
from typing import List, Dict, Any
from app.utils.embedding_utils import explode_embeddings
from app.stages.feature_engineering_stage import calculate_area_similarity
from app.utils.strings_clean_utils import clean_area_atuacao

def predict_rank_for_vaga(df_candidates: pd.DataFrame, vaga_id: int, top_n: int = 5, 
                         model_path: str = "../models/lgbm_ranker.pkl",
                         pipeline_path: str = "../models/feature_pipeline.joblib") -> Dict[str, Any]:
    """
    Faz o predict e retorna o ranking de candidatos para uma vaga específica.
    
    Args:
        df_candidates: DataFrame com os candidatos a serem ranqueados
        vaga_id: ID da vaga para filtrar os candidatos
        top_n: Número de candidatos a retornar no ranking
        model_path: Caminho para o modelo treinado
        pipeline_path: Caminho para o pipeline de features
        
    Returns:
        Dicionário com o ID da vaga e a lista de candidatos ranqueados
    """
    try:
        # Filtra candidatos para a vaga específica
        df_vaga = df_candidates[df_candidates["codigo_vaga"] == vaga_id].copy()
        
        if df_vaga.empty:
            logger.warning(f"Nenhum candidato encontrado para a vaga {vaga_id}.")
            return {"vaga_id": vaga_id, "candidatos": []}
        
        # Carrega modelo e pipeline
        logger.info("Carregando modelo e pipeline...")
        model = load(model_path)
        pipe = load(pipeline_path)
        
        # Pré-processamento dos dados
        logger.info("Pré-processando os dados...")
        
        # 1. Expande embeddings
        df_vaga = explode_embeddings(df_vaga)
        
        # 2. Preenche valores faltantes
        for col in df_vaga.columns:
            if pd.api.types.is_numeric_dtype(df_vaga[col]):
                df_vaga[col] = df_vaga[col].fillna(-999)
            else:
                df_vaga[col] = df_vaga[col].fillna("Indefinido")
        
        # 3. Limpa áreas de atuação
        df_vaga['candidato_area_atuacao'] = clean_area_atuacao(df_vaga, 'candidato_area_atuacao')
        df_vaga['vaga_areas_atuacao_clean'] = clean_area_atuacao(df_vaga, 'vaga_areas_atuacao')
        
        # 4. Calcula similaridade de área
        df_vaga = calculate_area_similarity(df_vaga)


        # 5. Aplica transformações do pipeline
        logger.info("Aplicando transformações do pipeline...")
        X_processed = pipe.transform(df_vaga)
        
        
        # 6. Faz predições
        logger.info("Fazendo predições...")
        predictions = model.predict(X_processed)
        
        # 7. Cria DataFrame com resultados
        results_df = pd.DataFrame({
            'nome_candidato': df_vaga['nome_candidato'],
            'score': np.round(predictions, 4),
            'rank': predictions.argsort() + 1 ,

        })
        
        # 8. Ordena por score e pega top N candidatos
        top_candidates = (results_df.sort_values('rank', ascending=False)
                        .head(top_n)
                        .to_dict('records'))
        
        logger.success(f"Ranking gerado com sucesso para vaga {vaga_id}")
        return {
            "vaga_id": vaga_id,
            "candidatos": top_candidates
        }
        
    except Exception as e:
        logger.error(f"Erro ao gerar ranking para vaga {vaga_id}: {str(e)}")
        raise e

# Exemplo de uso:
vaga_id_exemplo = df_merged['codigo_vaga'].iloc[0]
resultado = predict_rank_for_vaga(df_merged, vaga_id_exemplo)
print(resultado)

2025-06-19 10:32:50.667 | INFO     | __main__:predict_rank_for_vaga:35 - Carregando modelo e pipeline...
2025-06-19 10:32:50.685 | INFO     | __main__:predict_rank_for_vaga:40 - Pré-processando os dados...
/tmp/ipykernel_23628/3109716850.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_vaga['vaga_areas_atuacao_clean'] = clean_area_atuacao(df_vaga, 'vaga_areas_atuacao')
2025-06-19 10:32:50.777 | INFO     | __main__:predict_rank_for_vaga:61 - Aplicando transformações do pipeline...
2025-06-19 10:32:50.784 | INFO     | __main__:predict_rank_for_vaga:66 - Fazendo predições...
2025-06-19 10:32:50.789 | SUCCESS  | __main__:predict_rank_for_vaga:82 - Ranking gerado com sucesso para vaga 4613


{'vaga_id': np.int32(4613), 'candidatos': [{'nome_candidato': 'Apollo da Conceição', 'score': 0.9102, 'rank': 4}, {'nome_candidato': 'Dr. Daniel Rocha', 'score': 0.975, 'rank': 3}, {'nome_candidato': 'Ravi Lucca Ribeiro', 'score': 0.9789, 'rank': 2}, {'nome_candidato': 'Henrique Ferreira', 'score': 1.2317, 'rank': 1}]}
